In [7]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

NotImplementedError: ignored

#Model Loading and Quantization

In [8]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [9]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [10]:
model_id = "dicta-il/dictalm-7b"

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto", trust_remote_code=True)

tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
def get_completion(query: str, model, tokenizer) -> str:
  device = "cuda:0"

  prompt_template = """
  {query}
  """
  prompt = prompt_template.format(query=query)

  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

  model_inputs = encodeds.to(device)


  generated_ids = model.generate(**model_inputs, max_new_tokens=10, do_sample=True, pad_token_id=tokenizer.eos_token_id)
  decoded = tokenizer.batch_decode(generated_ids)
  return (decoded[0])

In [12]:
result = get_completion(query="מה יהיה מזג האוויר מחר?", model=model, tokenizer=tokenizer)
print(result)


  מה יהיה מזג האוויר מחר?
  
ומה יקרה כשיסתיים?
יש כמה


# Dataset Prep and Prompt Function

In [13]:
import pandas as pd
import json

In [14]:
def extract_and_flatten_data(json_data):
    """
    Extracts and flattens data from JSON for DataFrame creation.

    :param json_data: Loaded JSON data.
    :return: List of dictionaries for DataFrame rows.
    """
    rows = []
    for entry in json_data["data"]:
        for answer_text in entry["answers"]["text"]:
            row = {
                "title": entry["title"],
                "context": entry["context"],
                "question": entry["question"],
                "text": answer_text
            }
            rows.append(row)
    return rows

In [15]:
train_file_path = '/content/train.json'
validation_file_path = '/content/validation.json'

In [16]:
# Open and load the JSON file
with open(train_file_path, 'r', encoding='utf-8') as file:
    json_data = json.load(file)

# Extracting and flattening data using the function
flattened_data = extract_and_flatten_data(json_data)

# Create DataFrame
train_df = pd.DataFrame(flattened_data)

In [17]:
train_df

,title,context,question,text
0,University_of_Notre_Dame,"מבחינה אדריכלית, לבית הספר יש אופי קתולי. בראש...",למי הופיעה לכאורה מריה הבתולה בשנת 1858 בלורד ...,סנט ברנדט סובירוס
1,University_of_Notre_Dame,"מבחינה אדריכלית, לבית הספר יש אופי קתולי. בראש...",מה יש מול הבניין הראשי של נוטרדאם?,פסל נחושת של ישו
2,University_of_Notre_Dame,"מבחינה אדריכלית, לבית הספר יש אופי קתולי. בראש...",בזיליקת הלב הקדוש בנוטרדאם נמצאת ליד איזה מבנה?,הבניין הראשי
3,University_of_Notre_Dame,"מבחינה אדריכלית, לבית הספר יש אופי קתולי. בראש...",מה יושב על ראש הבניין הראשי בנוטרדאם?,פסל זהב של מרים הבתולה
4,University_of_Notre_Dame,"כמו ברוב האוניברסיטאות האחרות, הסטודנטים של נו...",מתי החל ה-Scholastic Magazine of Notre Dame לפ...,ספטמבר 1876
...,...,...,...,...
52400,Kathmandu,שדה התעופה הבינלאומי העיקרי המשרת את קטמנדו ול...,מי מפעיל טיסות בין קטמנדו לאיסטנבול?,טורקיש איירליינס
52401,Kathmandu,"העיר מטרופולין קטמנדו (KMC), על מנת לקדם יחסים...","באיזו מדינה בארה""ב יצרה קטמנדו לראשונה מערכת י...",אורגון
52402,Kathmandu,"העיר מטרופולין קטמנדו (KMC), על מנת לקדם יחסים...",איך נודע בעבר יאנגון?,רנגון
52403,Kathmandu,"העיר מטרופולין קטמנדו (KMC), על מנת לקדם יחסים...",עם איזו עיר ביילורוסית יש לקטמנדו מערכת יחסים?,מינסק


In [19]:
# Open and load the JSON file
with open(validation_file_path, 'r', encoding='utf-8') as file:
    json_data = json.load(file)

# Extracting and flattening data using the function
flattened_data = extract_and_flatten_data(json_data)

# Create DataFrame
validation_df = pd.DataFrame(flattened_data)

In [20]:
validation_df

,title,context,question,text
0,Super_Bowl_50,סופרבול 50 היה משחק כדורגל אמריקאי כדי לקבוע א...,איזו קבוצת NFL ייצגה את ה-AFC בסופרבול 50?,דנבר ברונקוס
1,Super_Bowl_50,סופרבול 50 היה משחק כדורגל אמריקאי כדי לקבוע א...,איזו קבוצת NFL ייצגה את ה-AFC בסופרבול 50?,דנבר ברונקוס
2,Super_Bowl_50,סופרבול 50 היה משחק כדורגל אמריקאי כדי לקבוע א...,איזו קבוצת NFL ייצגה את ה-AFC בסופרבול 50?,דנבר ברונקוס
3,Super_Bowl_50,סופרבול 50 היה משחק כדורגל אמריקאי כדי לקבוע א...,היכן התקיים סופרבול 50?,"סנטה קלרה, קליפורניה"
4,Super_Bowl_50,סופרבול 50 היה משחק כדורגל אמריקאי כדי לקבוע א...,היכן התקיים סופרבול 50?,אצטדיון ליווי
...,...,...,...,...
20480,Force,"לכוח הפאונד יש מקבילה מטרי, בשימוש פחות נפוץ מ...",מה הכוונה לפעמים לכוח הקילוגרם?,קילפונד
20481,Force,"לכוח הפאונד יש מקבילה מטרי, בשימוש פחות נפוץ מ...",מה הכוונה לפעמים לכוח הקילוגרם?,קילפונד
20482,Force,"לכוח הפאונד יש מקבילה מטרי, בשימוש פחות נפוץ מ...",מה הכוונה לפעמים לכוח הקילוגרם?,קילפונד
20483,Force,"לכוח הפאונד יש מקבילה מטרי, בשימוש פחות נפוץ מ...",מה הכוונה לפעמים לכוח הקילוגרם?,קילפונד


In [21]:
validation_df.drop_duplicates(inplace=True)

In [22]:
validation_df

,title,context,question,text
0,Super_Bowl_50,סופרבול 50 היה משחק כדורגל אמריקאי כדי לקבוע א...,איזו קבוצת NFL ייצגה את ה-AFC בסופרבול 50?,דנבר ברונקוס
3,Super_Bowl_50,סופרבול 50 היה משחק כדורגל אמריקאי כדי לקבוע א...,היכן התקיים סופרבול 50?,"סנטה קלרה, קליפורניה"
4,Super_Bowl_50,סופרבול 50 היה משחק כדורגל אמריקאי כדי לקבוע א...,היכן התקיים סופרבול 50?,אצטדיון ליווי
5,Super_Bowl_50,סופרבול 50 היה משחק כדורגל אמריקאי כדי לקבוע א...,איזו קבוצת NFL זכתה בסופרבול 50?,דנבר ברונקוס
8,Super_Bowl_50,סופרבול 50 היה משחק כדורגל אמריקאי כדי לקבוע א...,באיזה צבע נעשה שימוש כדי להדגיש את יום השנה ה-...,זהב
...,...,...,...,...
20473,Force,הקשר בין כוחות מקרוסקופיים לא שמרניים לכוחות ש...,מהו חוק התרמודינמיקה הקשור לחילופי חום במערכת ...,החוק השני של התרמודינמיקה
20474,Force,הקשר בין כוחות מקרוסקופיים לא שמרניים לכוחות ש...,מה גורם לשינויי אנרגיה במערכת סגורה?,כוחות לא שמרניים
20478,Force,"לכוח הפאונד יש מקבילה מטרי, בשימוש פחות נפוץ מ...",מהו המונח המטרי פחות בשימוש מהניוטון?,כוח הקילוגרם (
20479,Force,"לכוח הפאונד יש מקבילה מטרי, בשימוש פחות נפוץ מ...",מה הכוונה לפעמים לכוח הקילוגרם?,קילפונד


In [23]:
def generate_prompt(data_point):

    text = f'### הקשר:\n{data_point["context"]}\n\n'
    text += f'### שאלה:\n{data_point["question"]}'

    return text

In [24]:
# add the "prompt" column in the dataset
text_column = [generate_prompt(data_point) for index, data_point in train_df.iterrows()]
train_df["prompt"] = text_column

text_column = [generate_prompt(data_point) for index, data_point in validation_df.iterrows()]
validation_df["prompt"] = text_column

In [29]:
from datasets import Dataset

In [30]:
def tokenize_function(examples):
    return tokenizer(examples["prompt"])

In [31]:
train_dataset = Dataset.from_pandas(train_df)
train_dataset = train_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/52405 [00:00<?, ? examples/s]

In [32]:
val_dataset = Dataset.from_pandas(validation_df)
val_dataset = val_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/9710 [00:00<?, ? examples/s]

#Model Prep for LORA training using PEFT



In [34]:
from peft import prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=False)

In [35]:
model

MegatronGPTForCausalLM(
  (megatron_gpt): MegatronGPTModel(
    (embed_in): Embedding(56064, 4096)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x MegatronGPTLayer(
        (input_layernorm): MegatronGPTLayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): MegatronGPTLayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (self_attention): MegatronGPTAttention(
          (rotary_emb): MegatronGPTRotaryEmbedding()
          (query_key_value): Linear4bit(in_features=4096, out_features=12288, bias=True)
          (dense): Linear4bit(in_features=4096, out_features=4096, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): MegatronGPTMLP(
          (dense_h_to_4h): Linear4bit(in_features=4096, out_features=10880, bias=True)
          (dens

In [36]:
import bitsandbytes as bnb
def find_all_linear_names(model):
  cls = bnb.nn.Linear4bit #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
  lora_module_names = set()
  for name, module in model.named_modules():
    if isinstance(module, cls):
      names = name.split('.')
      lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names: # needed for 16-bit
      lora_module_names.remove('lm_head')
  return list(lora_module_names)

In [37]:
modules = find_all_linear_names(model)
print(modules)

['dense_h_to_4h', 'dense', 'query_key_value', 'dense_4h_to_h']


In [38]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=modules,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

In [39]:
trainable, total = model.get_nb_trainable_parameters()
print(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.4f}%")

Trainable: 13959168 | total: 5474381824 | Percentage: 0.2550%


#Training

In [40]:
from huggingface_hub import notebook_login
notebook_login()

In [41]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [42]:
!pip install -q trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.8/100.8 kB 12.7 MB/s eta 0:00:00


In [43]:
import transformers

from trl import SFTTrainer

tokenizer.pad_token = tokenizer.eos_token
torch.cuda.empty_cache()

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    dataset_text_field="prompt",
    peft_config=lora_config,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=0.03,
        max_steps=100,
        learning_rate=2e-4,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        save_strategy="epoch",
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:194: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/52405 [00:00<?, ? examples/s]

Map:   0%|          | 0/9710 [00:00<?, ? examples/s]

In [44]:
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,5.715100
2,5.939000
3,5.454100
4,4.893000
5,4.513600
6,4.283000
7,4.000600
8,4.614900
9,4.159500
10,3.909800


TrainOutput(global_step=100, training_loss=3.591363215446472, metrics={'train_runtime': 295.3483, 'train_samples_per_second': 1.354, 'train_steps_per_second': 0.339, 'total_flos': 2172739964313600.0, 'train_loss': 3.591363215446472, 'epoch': 0.01})

In [45]:
model.push_to_hub("dictalm_squad_finetuned_test")
tokenizer.push_to_hub("dictalm_squad_finetuned_test")

adapter_model.safetensors:   0%|          | 0.00/55.9M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/omriabo/dictalm_squad_finetuned_test/commit/fb555540d9bd3aeb1c03ed1f3b873ae62a55f811', commit_message='Upload tokenizer', commit_description='', oid='fb555540d9bd3aeb1c03ed1f3b873ae62a55f811', pr_url=None, pr_revision=None, pr_num=None)